# WebSraping Candidatos Governadores Bahai 208

**Objetivo**: Gerar `.csv` de todos os dados de governadores apartir de 3 fontes
+ tse
+ estadao
+ gazeta do povo

# Import and Constants

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os 

# Snippets

In [ ]:
def convert_long_comment_in_string(long_txt):
    """
    Converter uma lsita grande dos attrs que queremos filtrar, pois selecionei manualmente
    """
    return list(map(lambda x: x.strip(), long_txt.split('\n')[1:-1] ))

In [ ]:
def filter_dict_deep_by_keys(adict, list_keys):
    """
    Goal: Filter some Keys of dict even his keys is a sub.key
    BR: Filtrar seomente alguns atributos do JSON, mesmo que seja um Objeto dentro de outro
    Baseado em: https://thispointer.com/python-filter-a-dictionary-by-conditions-on-keys-or-values/
    @list_keys vem dea funcao : 'convert_long_comment_in_string'
    """
    newDict = dict()
    for (key, value) in adict.items():
        if key in list_keys:
            newDict[key] = value
    return newDict            

In [110]:
def request_get(url, lib_parser = 'html.parser'):
    """
    lib_paeser : 'html.parser' // 
    return BeautifulSoupPage
    """
    response = requests.get(url)
    if(response.status_code != 200):
        print(url, 'Erro Code')
        quit()
    return BeautifulSoup(response.text, lib_parser)

# One Governador

Será reunido os dados de um único candidato a governador **1 Turno** ``João Henrique Durval Carneiro - PTB 28``

use https://jsonformatter.org/json-viewer para ver os dados

## TSE Data

In [101]:
tse_url_1 = 'https://divulgacandcontas.tse.jus.br/divulga/rest/v1/candidatura/buscar/2018/BA/2022802018/candidato/50000622082'
tse_url_2 = 'https://divulgacandcontas.tse.jus.br/divulga/rest/v1/prestador/consulta/2022802018/2018/BA/3/28/28/50000622082'

In [111]:
tse1 = requests.get(tse_url_1).json()
tse2 = requests.get(tse_url_2).json()

In [105]:
list_governador_keys_tse1 = """
    id
    nomeUrna
    numero
    nomeCompleto
    descricaoSexo
    descricaoCorRaca
    grauInstrucao
    ocupacao
    gastoCampanha1T
    gastoCampanha2T
    sgUfNascimento
    nomeMunicipioNascimento
    localCandidatura
    fotoUrl
    descricaoTotalizacao
    nomeColigacao
    composicaoColigacao
    cargo
    totalDeBens
    partido
    descricaoNaturalidade
"""

In [116]:
list_governador_keys_tse2 = """
    nomePartido
    siglaPartido
    nrCandidato
    despesas
    concentracaoDespesas
    rankingDoadores
    rankingFornecedores
    dividaCampanha
    sobraFinanceira
    dadosConsolidados
"""

In [117]:
# Lista dos atributos a serem filtrados do JON, pois o JSON é muito grande
# Escolhidos arbitrariamente de acordco com a proposta
l1 = convert_long_comment_in_string(list_governador_keys_tse1) 
l2 = convert_long_comment_in_string(list_governador_keys_tse2)

In [107]:
tse1_re = filter_dict_deep_by_keys(tse1, l1)
tse1_re

{'id': 50000622082,
 'nomeUrna': 'JOÃO HENRIQUE DURVAL CARNEIRO',
 'numero': 28,
 'nomeCompleto': 'JOÃO HENRIQUE DE BARRADAS CARNEIRO',
 'descricaoSexo': 'MASC.',
 'descricaoCorRaca': 'PARDA',
 'grauInstrucao': 'Superior completo',
 'ocupacao': 'Economista',
 'gastoCampanha1T': 14000000,
 'gastoCampanha2T': 7000000,
 'sgUfNascimento': 'BA',
 'nomeMunicipioNascimento': 'FEIRA DE SANTANA',
 'localCandidatura': 'BAHIA',
 'fotoUrl': 'https://divulgacandcontas.tse.jus.br/candidaturas/oficial/2018/BR/BA/2022802018/50000622082/foto_1534294033690.jpg',
 'descricaoTotalizacao': 'Não eleito',
 'nomeColigacao': 'BAHIA ACIMA DE TUDO, DEUS ACIMA DE TODOS',
 'composicaoColigacao': 'PRTB / PSL',
 'cargo': {'codigo': 3,
  'sigla': None,
  'nome': 'Governador',
  'codSuperior': 0,
  'titular': True,
  'contagem': 0},
 'totalDeBens': 1502885.55,
 'partido': {'numero': 28,
  'sigla': 'PRTB',
  'nome': 'Partido Renovador Trabalhista Brasileiro'},
 'descricaoNaturalidade': 'BA-FEIRA DE SANTANA'}

In [109]:
tse1_re['cargo'] = tse1_re['cargo']['nome']
tse1_re['sigla'] = tse1_re['partido']['sigla']
tse1_re['partido'] = tse1_re['partido']['nome']
tse1_re

{'id': 50000622082,
 'nomeUrna': 'JOÃO HENRIQUE DURVAL CARNEIRO',
 'numero': 28,
 'nomeCompleto': 'JOÃO HENRIQUE DE BARRADAS CARNEIRO',
 'descricaoSexo': 'MASC.',
 'descricaoCorRaca': 'PARDA',
 'grauInstrucao': 'Superior completo',
 'ocupacao': 'Economista',
 'gastoCampanha1T': 14000000,
 'gastoCampanha2T': 7000000,
 'sgUfNascimento': 'BA',
 'nomeMunicipioNascimento': 'FEIRA DE SANTANA',
 'localCandidatura': 'BAHIA',
 'fotoUrl': 'https://divulgacandcontas.tse.jus.br/candidaturas/oficial/2018/BR/BA/2022802018/50000622082/foto_1534294033690.jpg',
 'descricaoTotalizacao': 'Não eleito',
 'nomeColigacao': 'BAHIA ACIMA DE TUDO, DEUS ACIMA DE TODOS',
 'composicaoColigacao': 'PRTB / PSL',
 'cargo': 'Governador',
 'totalDeBens': 1502885.55,
 'partido': 'Partido Renovador Trabalhista Brasileiro',
 'descricaoNaturalidade': 'BA-FEIRA DE SANTANA',
 'sigla': 'PRTB'}

In [118]:
tse2_re = filter_dict_deep_by_keys(tse2, l2)
tse2_re

{'nomePartido': 'Partido Renovador Trabalhista Brasileiro',
 'siglaPartido': 'PRTB',
 'nrCandidato': 28,
 'dadosConsolidados': {'totalRecebido': 1500,
  'qtdRecebido': 2,
  'percentualRecebido': None,
  'totalFinanceiro': 1000,
  'qtdFinanceiro': 1,
  'percentualFinanceiro': 66.67,
  'totalEstimados': 500,
  'qtdEstimados': 1,
  'percentualEstimados': 33.33,
  'totalReceitaPF': 1500,
  'qtdReceitaPF': 2,
  'percentualReceitaPF': 100.0,
  'totalReceitaPJ': 0,
  'qtdReceitaPJ': 0,
  'percentualReceitaPJ': 0,
  'totalPartidos': 0,
  'qtdPartidos': 0,
  'percentualPartidos': 0,
  'totalInternet': 0,
  'qtdInternet': 0,
  'percentualInternet': 0,
  'totalRoni': 0,
  'qtdRoni': 0,
  'percentualRoni': 0,
  'totalProprios': 0,
  'qtdProprios': 0,
  'percentualProprios': 0,
  'totalReceitaOutCand': 0,
  'qtdReceitaOutCand': 0,
  'percentualReceitaOutCand': 0,
  'totalDoacaoFcc': 0,
  'qtdDoacaoFcc': 0,
  'percentualDoacaoFcc': 0,
  'sqPrestadorConta': None,
  'sqEntregaPrestacao': None,
  'grap

In [123]:
# Por todos os attrs de despesas pra fora
for el in list(tse2_re['despesas'].keys()):
    tse2_re[el] = tse2_re['despesas'][el]
    
# Por todos os attrs de dadosConsolidados pra fora
for el in list(tse2_re['dadosConsolidados'].keys()):
    tse2_re[el] = tse2_re['dadosConsolidados'][el]
    
tse2_re.pop('despesas', None)
tse2_re.pop('dadosConsolidados', None)

KeyError: 'despesas'

In [131]:
tse2_re['concentracaoDespesasR'] = sum(map(lambda x: x['valor'], tse2_re['concentracaoDespesas']))
tse2_re['rankingDoadoresR'] = sum(map(lambda x: x['valor'], tse2_re['rankingDoadores']))
tse2_re['rankingFornecedoresR'] = sum(map(lambda x: x['valor'], tse2_re['rankingFornecedores']))

tse2_re.pop('concentracaoDespesas', None)
tse2_re.pop('rankingDoadores', None)
tse2_re.pop('rankingFornecedores', None)

In [132]:
tse2_re

{'nomePartido': 'Partido Renovador Trabalhista Brasileiro',
 'siglaPartido': 'PRTB',
 'nrCandidato': 28,
 'dividaCampanha': None,
 'sobraFinanceira': None,
 'limiteDeGasto1T': 14000000.0,
 'limiteDeGasto2T': 7000000.0,
 'valorLimiteDeGastos': 14000000.0,
 'totalDespesasContratadas': 1000.0,
 'totalDespesasPagas': 1000.0,
 'doacoesOutrosCandidatosPartigos': 0.0,
 'fundosPartidarios': 0.0,
 'fundoEspecial': 0.0,
 'outrosRecursos': 1000.0,
 'financeiras': 0.0,
 'estimaveis': 0.0,
 'totalRecebido': 1500,
 'qtdRecebido': 2,
 'percentualRecebido': None,
 'totalFinanceiro': 1000,
 'qtdFinanceiro': 1,
 'percentualFinanceiro': 66.67,
 'totalEstimados': 500,
 'qtdEstimados': 1,
 'percentualEstimados': 33.33,
 'totalReceitaPF': 1500,
 'qtdReceitaPF': 2,
 'percentualReceitaPF': 100.0,
 'totalReceitaPJ': 0,
 'qtdReceitaPJ': 0,
 'percentualReceitaPJ': 0,
 'totalPartidos': 0,
 'qtdPartidos': 0,
 'percentualPartidos': 0,
 'totalInternet': 0,
 'qtdInternet': 0,
 'percentualInternet': 0,
 'totalRoni': 0

In [133]:
z = {**tse1_re, **tse2_re}
z

{'id': 50000622082,
 'nomeUrna': 'JOÃO HENRIQUE DURVAL CARNEIRO',
 'numero': 28,
 'nomeCompleto': 'JOÃO HENRIQUE DE BARRADAS CARNEIRO',
 'descricaoSexo': 'MASC.',
 'descricaoCorRaca': 'PARDA',
 'grauInstrucao': 'Superior completo',
 'ocupacao': 'Economista',
 'gastoCampanha1T': 14000000,
 'gastoCampanha2T': 7000000,
 'sgUfNascimento': 'BA',
 'nomeMunicipioNascimento': 'FEIRA DE SANTANA',
 'localCandidatura': 'BAHIA',
 'fotoUrl': 'https://divulgacandcontas.tse.jus.br/candidaturas/oficial/2018/BR/BA/2022802018/50000622082/foto_1534294033690.jpg',
 'descricaoTotalizacao': 'Não eleito',
 'nomeColigacao': 'BAHIA ACIMA DE TUDO, DEUS ACIMA DE TODOS',
 'composicaoColigacao': 'PRTB / PSL',
 'cargo': 'Governador',
 'totalDeBens': 1502885.55,
 'partido': 'Partido Renovador Trabalhista Brasileiro',
 'descricaoNaturalidade': 'BA-FEIRA DE SANTANA',
 'sigla': 'PRTB',
 'nomePartido': 'Partido Renovador Trabalhista Brasileiro',
 'siglaPartido': 'PRTB',
 'nrCandidato': 28,
 'dividaCampanha': None,
 'sob

## Gazeta do Povo

In [19]:
url_gazeta = 'https://especiais.gazetadopovo.com.br/eleicoes/2018/candidatos/ba/governador/joao-henrique-durval-carneiro/'

In [26]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import os 
response = requests.get(url_gazeta)
if(response.status_code != 200):
    print(url, 'Erro Code')
    quit()
soup_page = BeautifulSoup(response.text, 'html.parser')

In [39]:
# <section class="tabela-candidato">
section_html = soup_page.find('section', {"class": "tabela-candidato"})

section_desc = section_html.find('p').get_text()
section_winn = section_html.find('span', {"class": "label label-default"}).get_text()
sextion_link_cityes = section_html.find_all('tr')[-1].td.a['href']


In [40]:
print(section_desc)
print()
print(section_winn)
print()
print(sextion_link_cityes)

Candidato ao cargo de Governador na Bahia pelo PRTB, João Henrique Durval Carneiro obteve 38.960 votos totalizados (0,58% dos votos válidos) mas não foi eleito nas Eleições 2018.

Não foi eleito

https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/governador-candidato-joao-henrique-durval-carneiro/


In [ ]:
# <div class="content-sidebar bg-cinza-claro">

In [43]:
section_html_bio = soup_page.find('div', {"class": "content-sidebar bg-cinza-claro"})

section_bio_ultra_desc = section_html_bio.find(
    'div', {'class': 'bio'}).find('div', {'class': 'text'}).find('p').get_text()

# <table class="table">
section_table = section_html_bio.find('table')

bio_dict = dict()
section_bio_tr = section_table.find_all('tr')
for el in section_bio_tr:
    bio_dict[el.th.get_text()] = el.td.get_text()

In [44]:
print(section_bio_ultra_desc)
print()
print(bio_dict)

João Henrique de Barradas Carneiro é candidato ao governo da Bahia pelo PRTB. Ele já foi vereador de Salvador (1989-1994), deputado estadual pela Bahia (1995-2004) e prefeito de Salvador. Como prefeito, foi eleito em 2004 e depois reeleito em 2008. É filho do ex-governador baiano João Durval Carneiro.  É economista formado pela Universidade Federal da Bahia (UFBA) e tem pós-graduação em Desenvolvimento Econômico pela Université du Québec à Montréal, do Canadá. 

{'Nome na urna:': 'João Henrique Durval Carneiro', 'Sexo:': 'Masculino', 'Idade:': '62 anos', 'Data de nascimento:': '19/06/1959', 'Ocupação:': 'Economista', 'Grau de instrução:': 'Superior completo', 'Estado Civil:': 'Casado(a)', 'Cor/Raça:': 'Parda', 'Município de nascimento:': 'Feira de Santana-BA'}


In [ ]:
# <div class="bio">
#                 <div class="text">
#                   <p>João Henrique de Barradas Carneiro é candidato ao governo da Bahia pelo PRTB. Ele já foi vereador de Salvador (1989-1994), deputado estadual pela Bahia (1995-2004) e prefeito de Salvador. Como prefeito, foi eleito em 2004 e depois reeleito em 2008. É filho do ex-governador baiano João Durval Carneiro. É economista formado pela Universidade Federal da Bahia (UFBA) e tem pós-graduação em Desenvolvimento Econômico pela Université du Québec à Montréal, do Canadá. </p>
#                 </div>
#               </div>

## Dados da cidade

In [45]:
sextion_link_cityes

'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/governador-candidato-joao-henrique-durval-carneiro/'

In [46]:
response = requests.get(sextion_link_cityes)
if(response.status_code != 200):
    print(url, 'Erro Code')
    quit()
soup_page_cities = BeautifulSoup(response.text, 'html.parser')

In [48]:
# soup_page_cities

In [ ]:
#           <tbody>
#             <tr>
#               <td class="candidato">
#                 <a class="clearfix" href="https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/mortugaba-ba/governador/">
#                   <div class="nome-partido">
#                     <h2 class="nome"> Mortugaba </h2>
#                   </div>
#                 </a>
#               </td>
#               <td class="votos">
#                 <span class="porcentagem"> 2,65% </span>
#                 <span class="qtd-votos">150 votos</span>
#               </td>
#             </tr>
#             <tr>
#               <td class="candidato">

In [56]:
list_tr = soup_page_cities.find('tbody').find_all('tr')
cities_dicts = []

for el in list_tr:
    city_dict = {}
    tds = el.find_all('td')
    city_dict['url'] = tds[0].find('a')['href']
    city_dict['name'] = tds[0].find('h2').get_text().strip()
    city_dict['percentage'] = tds[1].find('span', {'class': 'porcentagem'}).get_text().strip()
    city_dict['votes'] = tds[1].find('span', {'class': 'qtd-votos'}).get_text().split(' ')[0].strip()
    cities_dicts.append(city_dict)

# Formato de Siada
#     city_dict {'url': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/mortugaba-ba/governador/', 'name': 'Mortugaba', 'percentage': '2,65%', 'votes': '150'}

## Dados de uma cidade para governador

````
<table class="table table-resultados sans-serif">
<thead>
   <tr>
      <th>Candidato</th>
      <th class="text-right">Votos *</th>
   </tr>
</thead>
<tbody>
   <tr>
   
      <td class="candidato">
         <a class="clearfix" href="...">
            <figure class="thumb">
               <img src=".....jpg" alt="Candidato Rui Costa 13">        					
            </figure>
            <div class="nome-partido">
               <h2 class="nome">
                  Rui Costa <small class="label label-success">Eleito</small>        				
               </h2>
               <span class="partido">PT</span>
            </div>
         </a>
      </td>
      
      <td class="votos">
         <span class="porcentagem">
         70,35%        				</span>
         <span class="qtd-votos">3.986 votos</span>
      </td>
   </tr>
   <tr>
      <td class="candidato">
````

In [59]:
governador_para_cidade = cities_dicts[0]['url']
governador_para_cidade

'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/mortugaba-ba/governador/'

In [60]:
response = requests.get(governador_para_cidade)
if(response.status_code != 200):
    print(url, 'Erro Code')
    quit()
soup_page_governador_para_cidade = BeautifulSoup(response.text, 'html.parser')

In [66]:
list_tr_cidade_many_gov = soup_page_governador_para_cidade.find('tbody').find_all('tr')
city_by_governadores_dicts = []

for el in list_tr_cidade_many_gov:
    city_dict = {}
    tds = el.find_all('td')
    city_dict['url'] = tds[0].find('a')['href']
    city_dict['name'] = tds[0].find('h2').get_text().strip().replace('Eleito','')
    city_dict['percentage'] = tds[1].find('span', {'class': 'porcentagem'}).get_text().strip()
    city_dict['votes'] = tds[1].find('span', {'class': 'qtd-votos'}).get_text().split(' ')[0].strip()
    print(city_dict)
    break
    city_by_governadores_dicts.append(city_dict)

{'url': 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/governador-candidato-rui-costa/', 'name': 'Rui Costa ', 'percentage': '70,35%', 'votes': '3.986'}


## Dados de uma cidade

````
<dl class="perfil-eleitores">
    <dt>Total de Eleitores:</dt>
    <dd>9.522</dd>
    <dt>Eleitorado Feminino:</dt>
    <dd>4.898 (51,44%)</dd>
    <dt>Eleitorado Masculino:</dt>
    <dd>4.624 (48,56%)</dd>
    <dt>Eleitorado com sexo não infomado:</dt>
    <dd>0 (0,00%)</dd>
    ...
````

In [67]:
xx = cities_dicts[0]['url']
xx

'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/mortugaba-ba/governador/'

In [73]:
xx.split('/')[:-2]

['https:',
 '',
 'especiais.gazetadopovo.com.br',
 'eleicoes',
 '2018',
 'resultados',
 'municipios-bahia',
 'mortugaba-ba']

In [74]:
'/'.join(xx.split('/')[:-2])

'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/mortugaba-ba'

In [75]:
response = requests.get('/'.join(xx.split('/')[:-2]))
if(response.status_code != 200):
    print(url, 'Erro Code')
    quit()
soup_page_a_city = BeautifulSoup(response.text, 'html.parser')

In [78]:
temp_page = soup_page_a_city.find('dl', {'class': 'perfil-eleitores'})
all_dt, all_dd = temp_page.find_all('dt'), temp_page.find_all('dd')

city_data_dict = {}
city_data_dict['totalVotes'] = all_dd[0].get_text().strip().replace('.','')
city_data_dict['mascVotes'] =  all_dd[1].get_text().strip().split(' ')[0].replace('.','')
city_data_dict['femnVotes'] = all_dd[2].get_text().strip().split(' ')[0].replace('.','')

city_data_dict

{'totalVotes': '9522', 'mascVotes': '4898', 'femnVotes': '4624'}

# Todos os Governadores

In [8]:
tse_all_candidates = 'https://divulgacandcontas.tse.jus.br/divulga/rest/v1/candidatura/listar/2018/BA/2022802018/3/candidatos'